<img src="assets/ErrorTypes.png" align="left" width="600" style="margin-right:15px;"/>

In [ ]:
from typing import TypedDict, Literal

class State(TypedDict):
    pass

In [ ]:
# Transient errors: Add a retry policy to automatically retry network issues and rate limits:
from langgraph.types import RetryPolicy

workflow.add_node(
    "search_documentation",
    search_documentation,
    retry_policy=RetryPolicy(max_attempts=3, initial_interval=1.0)
)


In [ ]:
# LLM-recoverable: Store the error in state and loop back so the LLM can see what went wrong and try again
from langgraph.types import Command


def execute_tool(state: State) -> Command[Literal["agent", "execute_tool"]]:
    try:
        result = run_tool(state['tool_call'])
        return Command(update={"tool_result": result}, goto="agent")
    except ToolError as e:
        # Let the LLM see what went wrong and try again
        return Command(
            update={"tool_result": f"Tool error: {str(e)}"},
            goto="agent"
        )

In [ ]:
# User-fixable: Pause and collect information from the user when needed (like account IDs, order numbers, or clarifications)
from langgraph.types import Command


def lookup_customer_history(state: State) -> Command[Literal["draft_response"]]:
    if not state.get('customer_id'):
        user_input = interrupt({
            "message": "Customer ID needed",
            "request": "Please provide the customer's account ID to look up their subscription history"
        })
        return Command(
            update={"customer_id": user_input['customer_id']},
            goto="lookup_customer_history"
        )
    # Now proceed with the lookup
    customer_data = fetch_customer_history(state['customer_id'])
    return Command(update={"customer_history": customer_data}, goto="draft_response")

In [ ]:
# Unexpected: Let them bubble up for debugging. Don’t catch what you can’t handle
def send_reply(state: EmailAgentState):
    try:
        email_service.send(state["draft_response"])
    except Exception:
        raise  # Surface unexpected errors